In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
import psycopg2

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  print(endpoint)
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:
db = load_db()

yeoreodigm-rds.cyjwviwnpg1s.ap-northeast-2.rds.amazonaws.com


In [ ]:
def load_db():
  endpoint = "yeoreodigm-rds.cyjwviwnpg1s.ap-northeast-2.rds.amazonaws.com"
  dbname = "yeoreodigm"
  user = "yeoreodigm_ai"
  password = "ai1208!"
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:
db = load_db()

##기존의 취향 계산함수

In [ ]:
def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  #1. retrieve from survey
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*22)
  theme_score = theme_score.reshape(1,-1)
  for place in survey_result:
    load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)*10
  #2. retrieve from place-like log
  load_like_log_sql = f'SELECT place_id FROM place_like WHERE member_id = {user_id};'
  cursor.execute(load_like_log_sql)
  user_like_log = cursor.fetchall()
  if user_like_log:
    user_like_log = list(map(lambda x: x[0],user_like_log))
    for place in user_like_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)*7
  
  #3. retrieve from place_click log
  load_click_log_sql = f'SELECT place_id FROM places_log WHERE member_id = {user_id};'
  cursor.execute(load_click_log_sql)
  user_click_log = cursor.fetchall()
  if user_click_log:
    user_click_log = list(map(lambda x: x[0],user_click_log))
    for place in user_click_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)
  print("calculate_tast DONE!!")
  print("user's taste : ",theme_score.reshape(1,-1))
  return theme_score.reshape(1,-1)

In [ ]:
#db에서 가져오는 함수를 만들
def retrieve_taste_db(user_id,db):
  cursor = db.cursor()
  sql = f"SELECT taste,last_update from user_taste where user_id={user_id}"
  cursor.execute(sql)
  result = cursor.fetchone()
  return result

In [ ]:
class user():
  def __init__(self, user_id):
    self.id=user_id

In [ ]:
test_user = user(184)

In [ ]:
db=load_db()

In [ ]:
result = retrieve_taste(test_user.id,db)

In [ ]:
result == None

True

In [ ]:
user_taste = calculate_taste(184,db)
user_taste

calculate_tast...
[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
calculate_tast DONE!!
user's taste :  [[ 41   9 124 106  41   8  43 188 182  61  37  27 166  87 104 104  45  27
  220  42   7  11]]


array([[ 41,   9, 124, 106,  41,   8,  43, 188, 182,  61,  37,  27, 166,
         87, 104, 104,  45,  27, 220,  42,   7,  11]])

In [ ]:
db.rollback()

In [ ]:
list(map(int, user_taste[0]))

list

In [ ]:
taste = list(user_taste[0])
cursor = db.cursor()
cursor.execute("INSERT INTO user_taste(user_id,taste) VALUES(%s, %s)",(184,list(map(int, user_taste[0])) ))

In [ ]:
db.commit()

In [ ]:
a = retrieve_taste(test_user.id,db)